Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [5]:
import re

In [6]:

df['hp'] = df['engine'].apply(lambda x: x.split()[0][:-2] if 'HP' in x.split()[0] else np.nan).astype('float')
df['engine_capacity'] =  df['engine'].apply(lambda x: re.findall('\d[.]\d{0,2}',re.findall('\d+[.]?\d*[\s]?l', x.lower())[0])[0] if re.search('\d+[.]?\d*[\s]?l', x.lower()) else np.nan)
df['cylinders'] =  df['engine'].apply(lambda x: re.findall('\d{1,2}',re.findall('\d{1,2}[\s]?cyl', x.lower())[0])[0] if re.search('\d{1,2}[\s]?cyl', x.lower()) else re.findall('\d{1,2}',re.findall('[v,h,i]\d{1,2}', x.lower())[0])[0] if re.search('[v,h,i]\d{1,2}', x.lower())  else np.nan)
df['transmission_type'] = df['transmission'].apply(lambda x : 'AT' if 'automatic' in x.lower() or 'a/t' in x.lower() or 'cvt' in x.lower() or 'dct' in x.lower() else 'MT' if 'm/t' in x.lower() or 'manual' in x.lower() else 'Unknown')
df['fuel_type']= df.apply(lambda row: re.findall('([a-zA-Z]+)\s*Fuel',row['engine'])[0] if re.search('([a-z]+)\s*fuel',row['engine'].lower()) and row['fuel_type'] == 'not supported' else row['fuel_type'], axis =1)
df['fuel_type'] = df['fuel_type'].replace(['E85 Flex Fuel', '-', 'Plug-In Hybrid'], ['Flex','Unknown','Hybrid'])
df['model_w\o_trim'] = df['model'].apply(lambda x: ' '.join(x.split()[:2]) if len(x.split())>2 else x.split()[0])
df['trim'] = df['model'].apply(lambda x: ' '.join(x.split()[2:]) if len(x.split())>2 else x.split()[1] if len(x.split())==2 else 'Unknown')

In [7]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,hp,engine_capacity,cylinders,transmission_type,model_w\o_trim,trim
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6,4,AT,Cooper S,Base
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9,8,AT,LS,V8
2,2,Chevrolet,Silverado 2500 LT,2002,136731,Flex,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,320.0,5.3,8,AT,Silverado 2500,LT
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0,5.0,8,Unknown,G90 5.0,Ultimate
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0,2.0,4,AT,Metris,Base


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 188533 non-null  int64  
 1   brand              188533 non-null  object 
 2   model              188533 non-null  object 
 3   model_year         188533 non-null  int64  
 4   milage             188533 non-null  int64  
 5   fuel_type          183450 non-null  object 
 6   engine             188533 non-null  object 
 7   transmission       188533 non-null  object 
 8   ext_col            188533 non-null  object 
 9   int_col            188533 non-null  object 
 10  accident           186081 non-null  object 
 11  clean_title        167114 non-null  object 
 12  price              188533 non-null  int64  
 13  hp                 155274 non-null  float64
 14  engine_capacity    181835 non-null  object 
 15  cylinders          174150 non-null  object 
 16  tr

In [19]:
df['temp'] = df['model'].apply(lambda x: len(x.split()))

In [20]:
df[df['temp'] > 3]['model'].value_counts()

model
E-Class E 350 4MATIC              1357
Rover Range Rover Sport HSE       1027
AMG G 63 Base                      994
Model Y Long Range                 798
C-Class C 300 4MATIC Sport         735
                                  ... 
Sportage Plug-In Hybrid X-Line       3
X5 xDrive 35i Sport Activity         2
IONIQ Plug-In Hybrid SEL             2
V60 T6 R-Design Platinum             2
Integra w/A-Spec Tech Package        1
Name: count, Length: 450, dtype: int64

In [14]:
[t for t in df['trim'] if len(t.split())>1]

['Premium Plus',
 'Rover P530 SE SWB',
 '63 S',
 '63 Base',
 '350 4MATIC',
 '63 Base',
 '300 4MATIC Sport',
 '53 Base',
 'Ti Sport',
 'Long Range',
 '3.8 Base',
 '300 4MATIC Sport',
 'Rover Sport HSE',
 'Base 4MATIC',
 'Special Edition',
 '550 4MATIC',
 'Rover Sport HSE',
 '4.0T Prestige',
 'Premium Plus',
 'S ALL4',
 'Laramie Mega Cab',
 '63 AMG',
 'AMG GT S',
 '45 Base 4MATIC',
 'Coupe Competition',
 'L Limited',
 'Horn/Lone Star',
 '220 4MATIC',
 'Premium Plus',
 'SLE V6',
 'Red Sport 400',
 'Premium Plus',
 'Long Range',
 'King Ranch',
 'Coupe AWD',
 'Rover Sport HSE Dynamic',
 'Crew Cab',
 'King Ranch',
 'Laramie Mega Cab',
 '350 4MATIC',
 'Standard Range Plus',
 'Ti Sport',
 'Base 4MATIC',
 'Premium Plus',
 'Long Range',
 'Rover Evoque S',
 'Red Sport 400',
 '3.8 Track',
 'LT H/D Extended Cab',
 '63 S',
 'Rover Sport Supercharged',
 'AMG C 63 S',
 'NV3500 HD SL V8',
 '300 4MATIC',
 'GLE 350',
 'Rover 3.0L V6 Supercharged HSE',
 'Tech Premium Plus',
 '43 Base 4MATIC',
 'F Sport',
